In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import numpy as np
from astropy.io import ascii
from astropy import units as u
from lmfit.models import PolynomialModel, GaussianModel
from lmfit import Model as mod
from lmfit import minimize, Parameters, report_fit
import ccdproc as ccdp
from pathlib import Path
import os
import glob
import shutil

In [7]:
#os.makedirs("CObaselineplots", exist_ok=True)
os.makedirs("COplots", exist_ok=True)

In [2]:
# Baseline + Baseline Subtraction Graphs
# technically don't need to do this but it's okay, good practice...
# Load all files matching pattern
file_list = sorted(glob.glob('COSpectra/*.txt'))  # Assumes files are in the same folder as this script
# Process each file
for filename in file_list:
    filename_base = os.path.splitext(os.path.basename(filename))[0]
    data = pd.read_csv(filename, sep='\s+', header=None)
    # Convert columns to numeric just in case
    x = pd.to_numeric(data[0], errors='coerce').values
    y = pd.to_numeric(data[1], errors='coerce').values
    # Remove NaNs caused by any non-numeric values
    mask = ~np.isnan(x) & ~np.isnan(y)
    x = x[mask]
    y = y[mask]

    gooddata = np.where((x >= -20) & (x <= 30))
    x=x[gooddata]
    y=y[gooddata]
    plt.xlim([-20, 30])
    plt.ylim([-5, 10])
    plt.plot(x,y)
    plt.axhline(0, color='green')
    plt.xlabel('Velocity(km/s)')
    plt.ylabel('Brightness(lm)')
    plt.title(f'CO; {filename_base}')
    #plt.legend()
    plt.tight_layout()
    plt.savefig(f"COplots/{filename_base}.png")
    plt.close()
    
    # ### Polynomial Baseline Fit ###
    # poly_model = PolynomialModel(degree=7)
    # gooddata1 = list(np.where((x >= -20) & (x <= -10))[0])
    # gooddata2 = list(np.where((x >= 15) & (x <= 30))[0])
    # gooddata = gooddata1 + gooddata2

    # params = poly_model.guess(y[gooddata], x=x[gooddata])
    # result = poly_model.fit(y[gooddata], params, x=x[gooddata])
    # y_eval = poly_model.eval(result.params, x=x)

    # ### Plotting Original with Baseline ###
    # plt.figure(figsize=(7, 5))
    # plt.xlim([-20, 30])
    # plt.ylim([-5, 10])
    # plt.axhline(0, color='green')
    # plt.plot(x, y, label='Data')
    # plt.plot(x, y_eval, label='Polynomial Baseline', color='red', linestyle='--')
    # plt.xlabel('Velocity(km/s)')
    # plt.ylabel('Brightness(lm)')
    # plt.title(f'Baseline of COData; {filename_base}')
    # plt.legend()
    # plt.tight_layout()
    # plt.savefig(f"CObaselineplots/baseline_{filename_base}.png")
    # plt.close()

    # #baseline is already subtracted...this is unecessary
    # ### Subtract Baseline ###
    # y_sub = y - y_eval

    # ### Plotting Subtracted Data ###
    # plt.figure(figsize=(7, 5))
    # plt.xlim([-20, 30])
    # plt.ylim([-5, 10])
    # plt.axhline(0, color='green')
    # plt.plot(x, y, label='Data')
    # plt.plot(x, y_sub, label='Subtracted Polynomial Baseline', color='red', linestyle='--')
    # plt.xlabel('Velocity(km/s)')
    # plt.ylabel('Brightness(lm)')
    # plt.title(f'Subtracted Baseline of COData; {filename_base}')
    # plt.legend()
    # plt.tight_layout()
    # plt.savefig(f"CObaselineplots/subtracted_baseline_{filename_base}.png")
    # plt.close()
print("All 37 files processed.")

All 37 files processed.


In [11]:
print(file_list)

['COSpectra\\158375-19250_CO.txt', 'COSpectra\\158500-19250_CO.txt', 'COSpectra\\158500-19375_CO.txt', 'COSpectra\\158625-19375_CO.txt', 'COSpectra\\158750-19375_CO.txt', 'COSpectra\\158750-19500_CO.txt', 'COSpectra\\158875-19500_CO.txt', 'COSpectra\\158875-19625_CO.txt', 'COSpectra\\159000-19625_CO.txt', 'COSpectra\\159000-19750_CO.txt', 'COSpectra\\159125-19750_CO.txt', 'COSpectra\\159125-19875_CO.txt', 'COSpectra\\159250-19875_CO.txt', 'COSpectra\\159250-20000_CO.txt', 'COSpectra\\159375-20000_CO.txt', 'COSpectra\\159500-20000_CO.txt', 'COSpectra\\159500-20125_CO.txt', 'COSpectra\\159625-20125_CO.txt', 'COSpectra\\159625-20250_CO.txt', 'COSpectra\\159750-20250_CO.txt', 'COSpectra\\159750-20375_CO.txt', 'COSpectra\\159875-20375_CO.txt', 'COSpectra\\159875-20500_CO.txt', 'COSpectra\\160000-20500_CO.txt', 'COSpectra\\160000-20625_CO.txt', 'COSpectra\\160125-20625_CO.txt', 'COSpectra\\160250-20625_CO.txt', 'COSpectra\\160250-20750_CO.txt', 'COSpectra\\160375-20750_CO.txt', 'COSpectra\\1

In [20]:
# ## Specify the current directory name and the new name##
# current_name = "CObaselineplots"
# new_name = "COplots"
# # Rename the directory
# os.rename(current_name, new_name)
# print(f"Directory renamed from '{current_name}' to '{new_name}'")

Directory renamed from 'CObaselineplots' to 'COplots'


In [19]:
len(file_list)

37

In [5]:
### Measure RMS ###
# RMS red
file_list = sorted(glob.glob('COSpectra/*.txt')) 
# Store results
rms_data = []
for filename in file_list:
    # Load your data
    data = pd.read_csv(filename, sep='\s+', header=None)
    y = pd.to_numeric(data[1], errors='coerce').values
    # Remove NaNs caused by any non-numeric values
    mask = ~np.isnan(y)
    y = y[mask]
    #y_sub = y - y_eval
    gooddata1 = list(np.where((x >= -20) & (x <= -10))[0])
    gooddata2 = list(np.where((x >= 15) & (x <= 30))[0])
    gooddata = gooddata1 + gooddata2
    noise=np.std(y[gooddata]) * u.K
    # Save results
    rms_data.append({
        "filename": filename,
        "noise(K)": noise.value
    })
# Convert into pandas
df = pd.DataFrame(rms_data)
print(df)

                         filename  noise(K)
0   COSpectra\158375-19250_CO.txt       0.0
1   COSpectra\158500-19250_CO.txt       0.0
2   COSpectra\158500-19375_CO.txt       0.0
3   COSpectra\158625-19375_CO.txt       0.0
4   COSpectra\158750-19375_CO.txt       0.0
5   COSpectra\158750-19500_CO.txt       0.0
6   COSpectra\158875-19500_CO.txt       0.0
7   COSpectra\158875-19625_CO.txt       0.0
8   COSpectra\159000-19625_CO.txt       0.0
9   COSpectra\159000-19750_CO.txt       0.0
10  COSpectra\159125-19750_CO.txt       0.0
11  COSpectra\159125-19875_CO.txt       0.0
12  COSpectra\159250-19875_CO.txt       0.0
13  COSpectra\159250-20000_CO.txt       0.0
14  COSpectra\159375-20000_CO.txt       0.0
15  COSpectra\159500-20000_CO.txt       0.0
16  COSpectra\159500-20125_CO.txt       0.0
17  COSpectra\159625-20125_CO.txt       0.0
18  COSpectra\159625-20250_CO.txt       0.0
19  COSpectra\159750-20250_CO.txt       0.0
20  COSpectra\159750-20375_CO.txt       0.0
21  COSpectra\159875-20375_CO.tx

In [6]:
# Computing the Integral
def integral(vel, ta, bounds, deltav):
    mask = (vel >= bounds[0]) & (vel <= bounds[1])
    deltav = vel[1]-vel[0]
    return np.sum(ta[mask])*deltav
# Set the path to your ASCII files
file_list = sorted(glob.glob('COSpectra/*.txt'))
bounds = [-10, 15]
# Process each file
for filename in file_list:
    data = pd.read_csv(filename, sep='\s+', header=None)
    vel = pd.to_numeric(data[0], errors='coerce').values
    ta = pd.to_numeric(data[1], errors='coerce').values
    # Remove NaNs caused by any non-numeric values
    mask =  ~np.isnan(vel) & ~np.isnan(ta)
    vel = vel[mask]
    ta = ta[mask]
    deltav = vel[1]-vel[0]
    #print(deltav)

    # Compute integral
    result = integral(vel, ta, bounds, deltav)
    print(f"{filename}: Integral from {bounds[0]} to {bounds[1]} = {result}")

COSpectra\158375-19250_CO.txt: Integral from -10 to 15 = 6.314554253400019
COSpectra\158500-19250_CO.txt: Integral from -10 to 15 = 1.3130652069000042
COSpectra\158500-19375_CO.txt: Integral from -10 to 15 = 2.9624802027000094
COSpectra\158625-19375_CO.txt: Integral from -10 to 15 = 1.6875226317000056
COSpectra\158750-19375_CO.txt: Integral from -10 to 15 = 1.6875226317000056
COSpectra\158750-19500_CO.txt: Integral from -10 to 15 = 1.5189543723000047
COSpectra\158875-19500_CO.txt: Integral from -10 to 15 = 1.930758710700006
COSpectra\158875-19625_CO.txt: Integral from -10 to 15 = 10.243580902500034
COSpectra\159000-19625_CO.txt: Integral from -10 to 15 = 14.102822658900047
COSpectra\159000-19750_CO.txt: Integral from -10 to 15 = 14.102822658900047
COSpectra\159125-19750_CO.txt: Integral from -10 to 15 = 35.792952998100105
COSpectra\159125-19875_CO.txt: Integral from -10 to 15 = 35.792952998100105
COSpectra\159250-19875_CO.txt: Integral from -10 to 15 = 37.77086998950012
COSpectra\15925

In [7]:
print(deltav)

0.650190000000002
